# Домашнее задание № 2. Мешок слов

In [1]:
import pandas as pd
import razdel
import nltk
from nltk.corpus import stopwords
from razdel import tokenize as razdel_tokenize

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [3]:
data = pd.read_csv('labeled.csv')

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [4]:
data.toxic.value_counts(normalize=True)

toxic
0.0    0.66514
1.0    0.33486
Name: proportion, dtype: float64

In [5]:
train, test = train_test_split(data, stratify=data.toxic, test_size=0.1, shuffle=True)

In [6]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

### Векторизация с дефолтной токенизацией

In [20]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.3)
X = vectorizer.fit_transform(train.comment) 
X_test = vectorizer.transform(test.comment) 

In [21]:
y = train.toxic.values
y_test = test.toxic.values

In [22]:
clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')

In [23]:
clf.fit(X, y)

RandomForestClassifier(class_weight='balanced')

In [24]:
clf.classes_

array([0., 1.])

In [25]:
preds = clf.predict(X_test)

In [26]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.80      0.91      0.85       959
         1.0       0.75      0.55      0.64       483

    accuracy                           0.79      1442
   macro avg       0.78      0.73      0.74      1442
weighted avg       0.78      0.79      0.78      1442



### Векторизация с токенизацией razdel

Вариант 1: применяем razdel_tokenizer напрямую в качестве аргумента в TfidfVectorizer.
Токенизация происходит внутри TfidfVectorizer на этапе преобразования текста в векторы.

In [11]:
from razdel import tokenize as razdel_tokenize

In [12]:
def razdel_tokenizer(text):
    tokens = list(razdel_tokenize(text))
    return ' '.join([token.text.lower() for token in tokens])

In [13]:
vectorizer = TfidfVectorizer(tokenizer=razdel_tokenizer, min_df=10, max_df=0.3)
X = vectorizer.fit_transform(train.comment) 
X_test = vectorizer.transform(test.comment) 
y = train.toxic.values
y_test = test.toxic.values

/Users/dariachelnokova/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:
clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.50      0.62       959
         1.0       0.44      0.78      0.56       483

    accuracy                           0.59      1442
   macro avg       0.63      0.64      0.59      1442
weighted avg       0.69      0.59      0.60      1442



Вариант 2. Создаем новые столбцы "tokens" в тренировочном и тестовом наборах данных, содержащие кастомный токенизированный текст.
Преобразует уже токенизированный с razdel текст из столбцов tokens в векторы с помощью TfidfVectorizer.
При этом векторайзер видимо сам дополнительно токенизирует уже токенизированный текст своими механизмами.

In [15]:
# Токенизация с razdel (создание новых столбцов в обучающей и тестовой выборках, содержащих токенизированные с razdel тексты)
train['tokens'] = train['comment'].apply(razdel_tokenizer)
test['tokens'] = test['comment'].apply(razdel_tokenizer)

In [16]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.3)
X = vectorizer.fit_transform(train['tokens']) 
X_test = vectorizer.transform(test['tokens'])
y = train['toxic'].values
y_test = test['toxic'].values

In [17]:
clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.89      0.85       959
         1.0       0.74      0.61      0.67       483

    accuracy                           0.80      1442
   macro avg       0.78      0.75      0.76      1442
weighted avg       0.79      0.80      0.79      1442



### Оценка
Если задавать свой токенизатор на этапе обучения векторизатора как параметр, то качество модели сильно ухудшается.
Если токенизировать текст своим токенизатором перед подачей векторизатору и при этом явно не задавать параметр "tokenizer", то результат получается другой: модель справилась лучше, чем при первом варианте кастомной токенизации, и примерно также, как с дефолтной токенизацией, по всем основным метриками для обоих классов.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

### 1. CountVectorizer + MultinomialNB

In [72]:
!pip install gensim

  Using cached gensim-4.3.2-cp311-cp311-macosx_10_9_x86_64.whl (24.1 MB)

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [27]:
from gensim.utils import tokenize

In [10]:
data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [30]:
def gensim_tokenizer(text):
    return list(tokenize(text, lowercase=True))

In [33]:
count_vectorizer = CountVectorizer(tokenizer=gensim_tokenizer, min_df=4, max_df=0.8, max_features=5000, ngram_range=(1,2))
X = count_vectorizer.fit_transform(train.comment)
X_test = count_vectorizer.transform(test.comment) 
y = train.toxic.values
y_test = test.toxic.values

/Users/dariachelnokova/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [34]:
clf = MultinomialNB(alpha=1, fit_prior=False)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.87      0.87       937
         1.0       0.76      0.77      0.76       505

    accuracy                           0.83      1442
   macro avg       0.82      0.82      0.82      1442
weighted avg       0.83      0.83      0.83      1442



In [35]:
# Вероятности принадлежности к классу 1 (токсичные)
probs = clf.predict_proba(X_test)[:, 1]

# Создаем DataFrame с предсказаниями и вероятностями
predictions_df = pd.DataFrame({
    'text': test['comment'],  # Тексты из тестовой выборки
    'toxicity_prediction': preds,  # Предсказания
    'toxicity_probability': probs  # Вероятности
})

# Находим 10 самых токсичных текстов
top_10_toxic = predictions_df.sort_values(by='toxicity_probability', ascending=False).head(10)

In [36]:
pd.set_option('display.max_colwidth', None)
top_10_toxic

text  \
941                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Конечно, это резонансное событие требует отдельного треда для обсуждения и 19 тредов до этого шло обсуждение. Тред назывался нейтрально Рассстрел мусульман в новозеландской мечети . Но сегодня ОП, по совместительству борец за Арийскую Раssy и мамкин фашик назвал его в честь стрелка KEBAB REMOVER(англ. уничтожитель чурок ) и с утра пораньше решил в ОП-посте(заголовке треда) обозначить тред, как героизирующий(в первом треде) и оправдывающий(в третьем треде) терракт этого стрелка. Пик стронгли релейтед. 20 тред. В третьей строке заголовка ОП восхваляет и героизирует террориста, цитирую заголовок: KEBAB REMOVER тред 20 Продолжаем пиздеть тут. Слава Герою! Я репортил тред в форме репорта и в прикрепе в d , реакции никакой. Видимо, школомодеры сами симпатизируют нацизму и закрывают на это глаза. 21 тред. После того, как я пригрозил ему репортом, в этот раз мамкин фашик, убирает строку про героя и зачем-то вместо неё ставит нацистское приветствие дивизии СС Галичина. KEBAB REMOVER тред 21 Продолжаем пиздеть тут. Слава Украiнi

Почему-то toxicity_probability для некоторых текстов здесь 100 процентная, выглядит подозрительно. Я не поняла, с чем связана такая высокая степень уверенности модели

### 2. TfidfVectorizer + LogisticRegression

In [4]:
data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [5]:
stop_words_russian = stopwords.words('russian')

In [6]:
tfidfvectorizer = TfidfVectorizer(max_features=9000, min_df=4, max_df=0.5, ngram_range=(1, 2), stop_words=stop_words_russian)
X = tfidfvectorizer.fit_transform(train.comment)
X_test = tfidfvectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [7]:
clf = LogisticRegression(C=0.7, class_weight='balanced', max_iter=2000, multi_class="ovr")
clf.fit(X, y)

preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.90      0.87      0.88       962
         1.0       0.75      0.80      0.78       480

    accuracy                           0.85      1442
   macro avg       0.83      0.84      0.83      1442
weighted avg       0.85      0.85      0.85      1442



In [8]:
probs = clf.predict_proba(X_test)[:, 1]
predictions_df = pd.DataFrame({
    'text': test['comment'],  
    'toxicity_prediction': preds,
    'toxicity_probability': probs
})
top_10_toxic = predictions_df.sort_values(by='toxicity_probability', ascending=False).head(10)

In [9]:
pd.set_option('display.max_colwidth', None)
top_10_toxic

,text,toxicity_prediction,toxicity_probability
839,"Нахуй иди чмо ебаное, рот твой ебал. Говна поешь, быдло\n",1.0,0.994479
1214,"блядь, заткнись ты . я уже взлетаю нахуй.\n",1.0,0.984635
186,"Вы вот смеетесь, а что будет, когда у нас эти смешные хохлы закончатся?\n",1.0,0.983661
1401,соси как хохлы сосут хохол\n,1.0,0.978343
669,"У-ух, ненавижу, блядь, хохлов.\n",1.0,0.978301
1048,Замуж тебе надо барыня...\n,1.0,0.975246
1315,"Но Трамп и в правда дегенерат. Жаль, что этот клоун победил и мы теперь живем в пиздеце. Блять, если бы Клинтон сейчас была у власти, то в мире не было бы такого трэша. В рот ебал промыток с альт-райтом головного мозга, вы же нихуя своей головой думать не можете, животные, мемов про сжв насмотрелись теперь строите из себя дохуя консерваторов.\n",1.0,0.971841
780,потому что хохлы - пидорашки\n,1.0,0.971239
1050,"Блять, маргиналошизики, чего вас так тянет сюда? Пиздуйте в свой тред и там создавайте борду с ue b er marginal\n",1.0,0.961476
659,Нахуй ты высрался тогда тут? МНЕ НИНРАВИЦА ЯСКОЗАЛ ХРЯ\n,1.0,0.961430


### Оценка
И там, и там комментарии токсичные, но не совпадают из-за разных векторайзеров в первую очередь, наверное. У CountVectorizer – токсичные лонгриды про политику, у Tfidf – короткие аграссивные высказывания.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

##### Стоп-слова

In [7]:
stop_words = list(stopwords.words('russian'))
# почему-то эти слова появлялись в топе токсичности у нескольких моделей, поэтому добавила их тоже в список стоп-слов
additional_stop_words = stop_words + ["это", "тебе", "почему", "очень", "просто", "например", "стоит", "ещё", "года", "знаю", "время", "всё", "лет", "вообще"]
additional_stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [8]:
vectorizer = TfidfVectorizer(stop_words=additional_stop_words, max_features=9000, min_df=3, max_df=0.07)
X = vectorizer.fit_transform(train.comment) 
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

### LogisticRegression

In [144]:
lr = LogisticRegression(C=0.1, class_weight='balanced')
lr.fit(X, y)
preds = lr.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.90      0.89      0.89       952
         1.0       0.78      0.80      0.79       490

    accuracy                           0.86      1442
   macro avg       0.84      0.84      0.84      1442
weighted avg       0.86      0.86      0.86      1442



In [176]:
# Получение слов из векторизатора
feature_names = vectorizer.get_feature_names_out()

# Получение коэффициентов признаков из обученной модели
coef = lr.coef_[0]

# Сопоставление слов и их важности
feature_importance = dict(zip(feature_names, coef))

# Получение топа наиболее важных токсичных слов
top_toxic_words = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
top_toxic_words = [(word, coef) for word, coef in top_toxic_words][:10]
top_toxic_words

[('хохлы', 1.6098924657473772),
 ('хохлов', 1.4739131403732524),
 ('нахуй', 1.2322095208399824),
 ('блять', 1.1448430196455224),
 ('блядь', 1.067223912226172),
 ('пиздец', 1.0594777556942567),
 ('хуй', 0.9896100748796677),
 ('сука', 0.9717789569005622),
 ('лол', 0.9317705943435961),
 ('дебил', 0.9138525707614747)]

### DecisionTreeClassifier

In [177]:
dtc = DecisionTreeClassifier(max_depth=1200, class_weight='balanced')
dtc.fit(X, y)
preds = dtc.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.85      0.77      0.81       952
         1.0       0.62      0.73      0.67       490

    accuracy                           0.76      1442
   macro avg       0.73      0.75      0.74      1442
weighted avg       0.77      0.76      0.76      1442



In [174]:
# Получение слов из векторайзера
feature_names = vectorizer.get_feature_names_out()

# Получение важности слов из классификатора
feature_importance = dtc.feature_importances_

# Сопоставление слов и их значимости, затем отбор топа наиболее важных токсичных слов
top_toxic_words_dtc = [(feature_names[idx], importance) for idx, importance in enumerate(feature_importance)]
top_toxic_words_dtc = sorted(top_toxic_words, key=lambda x: x[1], reverse=True)[:10]
top_toxic_words_dtc

[('хохлы', 1.6098924657473772),
 ('хохлов', 1.4739131403732524),
 ('нахуй', 1.2322095208399824),
 ('блять', 1.1448430196455224),
 ('блядь', 1.067223912226172),
 ('пиздец', 1.0594777556942567),
 ('хуй', 0.9896100748796677),
 ('сука', 0.9717789569005622),
 ('лол', 0.9317705943435961),
 ('дебил', 0.9138525707614747)]

### MultinomialNB

In [9]:
nb = MultinomialNB(alpha=1, fit_prior=False)
nb.fit(X, y)
preds = nb.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.89      0.88       959
         1.0       0.78      0.74      0.76       483

    accuracy                           0.84      1442
   macro avg       0.83      0.82      0.82      1442
weighted avg       0.84      0.84      0.84      1442



In [10]:
# Получение слов из векторизатора
feature_names = vectorizer.get_feature_names_out()

# Получение логарифма вероятности слова для каждого класса из модели
log_probs = nb.feature_log_prob_

# Нахождение топа токсичных слов для класса 1 (токсичные комментарии)
class_1_log_probs = log_probs[1]  # Выбор логарифмов вероятностей для класса 1
word_probs = list(zip(class_1_log_probs, feature_names))  # Сопоставление логарифмов вероятностей и слов
top_toxic_words_nb = sorted(word_probs, reverse=True)[:10]  # Получение топ слов с наибольшими логарифмами вероятностей
top_toxic_words_nb

[(-6.090361257073701, 'хохлы'),
 (-6.277617279223223, 'хохлов'),
 (-6.2779892595985345, 'нахуй'),
 (-6.545303981054182, 'блять'),
 (-6.548194340983322, 'пиздец'),
 (-6.555814690165637, 'блядь'),
 (-6.556869541119329, 'хуй'),
 (-6.592087323495406, 'тред'),
 (-6.706077945007733, 'сука'),
 (-6.813682963053642, 'лол')]

### RandomForestClassifier

In [11]:
rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', max_depth=20)
rf.fit(X, y)

preds = rf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.87      0.64      0.74       959
         1.0       0.53      0.81      0.64       483

    accuracy                           0.70      1442
   macro avg       0.70      0.73      0.69      1442
weighted avg       0.76      0.70      0.71      1442



In [15]:
# Получение слов из векторизатора
feature_names = vectorizer.get_feature_names_out()

# Получение значений важности признаков из классификатора
importances = rf.feature_importances_ 

# я так поняла, в RandomForest нельзя получить значимость слов для отдельных классов, только в целом
# Сортировка индексов признаков по важности и выбор топа самых важных
top_indices = importances.argsort()[::-1][:5]

# Создание списка из топа важных слов
top_toxic_words_rf = [(feature_names[idx], importances[idx]) for idx in top_indices]
top_toxic_words_rf


[('хохлов', 0.02008077289313733),
 ('хохлы', 0.019607595071454577),
 ('нахуй', 0.017667217041182882),
 ('тред', 0.016225391255067802),
 ('блядь', 0.014238066756471204)]